

1.   Download the required video
2.   Extract audio
3. Transcribe audion to text and translate the text to a destination language.
4. Create and audio using the translated text and clon the voice from the original video
5. Standadize the audio and video and upload to google drive
6. Upload the links of the files fron google drive
5. Use the new audio and original video to perform lipsyn







In [3]:
# ───────────────────────────────────────────────
# 0 SETUP
# ───────────────────────────────────────────────
!pip -q install ffmpeg-python openai-whisper TTS==0.22.0 pydub librosa numpy==1.26.4 syncsdk
!pip install coqui-tts
!apt -qq install -y ffmpeg
!pip install syncsdk

!pip install -U openai-whisper
!sudo apt update && sudo apt install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0, <3.9; 0.0.10.3 Requires-Python >=3.6.0, <3.9; 0.0.11 Requires-Python >=3.6.0, <3.9; 0.0.12 Requires-Python >=3.6.0, <3.9; 0.0.13.1 Requires-Python >=3.6.0, <3.9; 0.0.13.2 Requires-Python >=3.6.0, <3.9; 0.0.14.1 Requires-Python >=3.6.0, <3.9; 0.0.15 Requires-Python >=3.6.0, <3.9; 0.0.15.1 Requires-Python >=3.6.0, <3.9; 0.0.9 Requires-Python >=3.6.0, <3.9; 0.0.9.1 Requires-Python >=3.6.0, <3.9; 0.0.9.2 Requires-Python >=3.6.0, <3.9; 0.0.9a10 Requires-Python >=3.6.0, <3.9; 0.0.9a9 Requires-Python >=3.6.0, <3.9; 0.1.0 Requires-Python >=3.6.0, <3.10; 0.1.1 Requires-Python >=3.6.0, <3.10; 0.1.2 Requires-Python >=3.6.0, <3.10; 0.1.3 Requires-Python >=3.6.0, <3.10; 0.10.

In [3]:
!pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.6 MB/s eta 0:00:00


In [6]:
!pip install -U ffmpeg-python

  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)


Download the file from youtube (https://www.youtube.com/watch?v=32NNOzC_BAA&ab_channel=BFHTechnikundInformatik) or upload already existing file

In [4]:
# ───────────────────────────────────────────────
# 1 Download/Upload original video file
# ───────────────────────────────────────────────

#Working directory & inputs
from pathlib import Path
import shutil, os, json

WORK_DIR = Path("/content/fr_to_en_wav2lip")
WORK_DIR.mkdir(parents=True, exist_ok=True)
print("Work dir:", WORK_DIR)

#@markdown **Choose your source:** set a YouTube **URL** or upload a file
USE_COLAB_UPLOAD = False  #@param {type:"boolean"}
SOURCE_VIDEO_URL = "https://www.youtube.com/watch?v=32NNOzC_BAA&ab_channel=BFHTechnikundInformatik"  #@param {type:"string"}

SRC_PATH = WORK_DIR / "source_video.mp4"

if USE_COLAB_UPLOAD:
    try:
        from google.colab import files
        up = files.upload()
        if up:
            up_name = list(up.keys())[0]
            src = WORK_DIR / up_name
            shutil.move(str(src), str(SRC_PATH))
            print("Uploaded to:", SRC_PATH)
        else:
            print("No file selected.")
    except Exception as e:
        print("Colab upload not available:", e)
else:
    if SOURCE_VIDEO_URL.strip():
        import yt_dlp as ydl
        ydl_opts = {
            "outtmpl": str(WORK_DIR / "yt_src.%(ext)s"),
            "format": "mp4/bestaudio/best",
            "merge_output_format": "mp4",
        }
        with ydl.YoutubeDL(ydl_opts) as y:
            info = y.extract_info(SOURCE_VIDEO_URL, download=True)
            out = y.prepare_filename(info)
        if not out.endswith(".mp4"):
            !ffmpeg -y -i "$out" -c copy "{str(SRC_PATH)}"
        else:
            os.rename(out, str(SRC_PATH))
        print("Downloaded to:", SRC_PATH)
    else:
        print("Please provide a URL or set USE_COLAB_UPLOAD=True and upload a file.")

assert SRC_PATH.exists(), f"Missing source video: {SRC_PATH}"
!ffprobe -hide_banner -loglevel error -show_streams -select_streams v:0 "$SRC_PATH"


Work dir: /content/fr_to_en_wav2lip
[youtube] Extracting URL: https://www.youtube.com/watch?v=32NNOzC_BAA&ab_channel=BFHTechnikundInformatik
[youtube] 32NNOzC_BAA: Downloading webpage
[youtube] 32NNOzC_BAA: Downloading android sdkless player API JSON
[youtube] 32NNOzC_BAA: Downloading tv client config
[youtube] 32NNOzC_BAA: Downloading player 65578ad1-main
[youtube] 32NNOzC_BAA: Downloading tv player API JSON
[youtube] 32NNOzC_BAA: Downloading web safari player API JSON


[youtube] 32NNOzC_BAA: Downloading m3u8 information
[info] 32NNOzC_BAA: Downloading 1 format(s): 96
[download] Sleeping 6.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 19
[download] Destination: /content/fr_to_en_wav2lip/yt_src.mp4
[download] 100% of   35.73MiB in 00:00:02 at 17.73MiB/s                
[FixupM3u8] Fixing MPEG-TS in MP4 container of "/content/fr_to_en_wav2lip/yt_src.mp4"
Downloaded to: /content/fr_to_en_wav2lip/source_video.mp4
[STREAM]
index=1
codec_name=h264
codec_long_name=H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10
profile=High
codec_type=video
codec_tag_string=avc1
codec_tag=0x31637661
width=1920
height=1080
coded_width=1920
coded_height=1080
closed_captions=0
has_b_frames=1
sample_aspect_ratio=1:1
display_aspect_ratio=16:9
pix_fmt=yuv420p
level=40
color_range=tv
color_space=bt709
color_transfer=bt709
color_primaries=bt709
chroma_location=left
field_order=unknown
refs=1
is_avc=true
nal_length_size=4
id=N/A
r_

In [9]:
# ───────────────────────────────────────────────
# 2️ EXTRACT CLEAN AUDIO (16kHz mono WAV)
# ───────────────────────────────────────────────
import ffmpeg, os

video_path = "/content/fr_to_en_wav2lip/source_video.mp4"
wav_path = "/content/fr_to_en_wav2lip/French_vid.wav"

(
    ffmpeg
    .input(video_path)
    .output(wav_path, ac=1, ar=16000, format='wav')
    .overwrite_output()
    .run(quiet=True)
)

print("Extracted WAV:", wav_path)

Extracted WAV: /content/fr_to_en_wav2lip/French_vid.wav


In [15]:
# ───────────────────────────────────────────────
# 4 Specify the path of the files
# ───────────────────────────────────────────────
video_path = "/content/fr_to_en_wav2lip/source_video.mp4"    # your French source video
wav_path   = "/content/fr_to_en_wav2lip/French_vid.wav"     # we'll create this if you didn't upload a WAV

In [16]:
# ───────────────────────────────────────────────
# 5 TRANSCRIBE + TRANSLATE (French ➜ English)
# ───────────────────────────────────────────────
import whisper
model = whisper.load_model("small")  # try "small", "medium" or "large-v3" if you need higher accuracy
result = model.transcribe(wav_path, task="translate")  # FR -> EN
english_text = result["text"].strip()
print(english_text[:600] + "...")
print("Chars:", len(english_text))

/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


What is the most important aspect of your career? This Master represents a logical continuity for my academic and professional career. I wanted to expand my skills and to specialize in the field of data science. What is the most important aspect of your career? Yes, my computer studies with the orientation of data engineering were closely linked to practice. I had the opportunity to work on several projects, whether individually or in a team, which allowed me to apply concrete concepts and concepts in class. These projects allowed me to see how the formal principles are articulated in real-wor...
Chars: 1105


In [ ]:
# ───────────────────────────────────────────────
# 5.1 TRANSCRIBE + TRANSLATE (French ➜ English), using "large-v3"
# ───────────────────────────────────────────────
import whisper
model = whisper.load_model("large-v3")  # try "small", "medium" or "large-v3" if you need higher accuracy
result = model.transcribe(wav_path, task="translate")  # FR -> EN
english_text = result["text"].strip()
print(english_text[:600] + "...")
print("Chars:", len(english_text))

In [12]:
# ───────────────────────────────────────────────
# 6 CLONE SPEAKER VOICE & SYNTHESIZE ENGLISH AUDIO
# ───────────────────────────────────────────────

from TTS.api import TTS
from pydub import AudioSegment
import textwrap, os

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")

def chunk_text(txt, max_len=350):
    # Simple sentence-ish chunking to avoid super long inputs
    chunks = []
    for para in txt.split("\n"):
        p = para.strip()
        if not p:
            continue
        chunks.extend(textwrap.wrap(p, width=max_len, break_long_words=False, replace_whitespace=False))
    return chunks

chunks = chunk_text(english_text, max_len=350)
os.makedirs("xtts_chunks", exist_ok=True)
speaker_ref = wav_path  # or path to a clean reference sample
out_paths = []

for i, chunk in enumerate(chunks, 1):
    outp = f"xtts_chunks/part_{i:03d}.wav"
    tts.tts_to_file(
        text=chunk,
        file_path=outp,
        speaker_wav=speaker_ref,
        language="en"
    )
    out_paths.append(outp)

# Concatenate chunks
combined = AudioSegment.silent(duration=0)
for p in out_paths:
    combined += AudioSegment.from_wav(p)

dub_wav = "French_vid_dub_en.wav"
combined.export(dub_wav, format="wav")
print("English dub created:", dub_wav, "Duration (s):", round(combined.duration_seconds, 2))


/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y


100%|██████████| 1.87G/1.87G [00:29<00:00, 63.1MiB/s]
4.37kiB [00:00, 5.45MiB/s]
361kiB [00:00, 40.7MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 33.6kiB/s]
100%|██████████| 7.75M/7.75M [00:00<00:00, 63.6MiB/s]
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance

English dub created: French_vid_dub_en.wav Duration (s): 79.09


In [14]:
# ───────────────────────────────────────────────
# 5️ STANDARDIZE MEDIA (MP4 + WAV)
# ───────────────────────────────────────────────
!ffmpeg -y -i "/content/fr_to_en_wav2lip/source_video.mp4" -c:v libx264 -pix_fmt yuv420p -profile:v baseline -level 3.1 -movflags +faststart -an French_vid_h264.mp4
!ffmpeg -y -i "/content/French_vid_dub_en.wav" -ac 1 -ar 16000 -c:a pcm_s16le French_vid_dub_en_16k.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Upload the standarized files to google drive to be able to get a link

In [4]:
# ───────────────────────────────────────────────
# 6️ GET DIRECT GOOGLE DRIVE LINKS
# ───────────────────────────────────────────────
import os
from google.colab import drive
drive.mount('/content/drive')

import shutil
SAVE_DIR = "/content/drive/MyDrive/sync_out"
os.makedirs(SAVE_DIR, exist_ok=True)

shutil.copy("French_vid_h264.mp4", f"{SAVE_DIR}/French_vid_h264.mp4")
shutil.copy("French_vid_dub_en_16k.wav", f"{SAVE_DIR}/French_vid_dub_en_16k.wav")
print("Files saved to:", SAVE_DIR)
print("Now go to Google Drive → MyDrive → sync_out → right-click each file → Share → Anyone with link (Viewer) → Copy link")

# Convert Google Drive view links → direct links
import re
video_view = input("Paste your video view link: ")
audio_view = input("Paste your audio view link: ")

def to_direct(url):
    m = re.search(r"/d/([^/]+)/", url)
    return f"https://drive.google.com/uc?export=download&id={m.group(1)}" if m else url

video_url = to_direct(video_view)
audio_url = to_direct(audio_view)
print("Direct video URL:", video_url)
print("Direct audio URL:", audio_url)

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: 'French_vid_h264.mp4'

In [10]:
video_url = "https://drive.google.com/file/d/1zx6l1LT75BxjEO84yCVqitNndxTXQECh/view?usp=drive_link"
audio_url = "https://drive.google.com/file/d/11l60h0PLuchpdhFChL14mv_sMjRj8UEl/view?usp=drive_link"

In [11]:
# ───────────────────────────────────────────────
# 7️ RUN LIPSYNC GENERATION (SYNC)
# ───────────────────────────────────────────────

import time
from sync import Sync
from sync.common import Audio, GenerationOptions, Video
from sync.core.api_error import ApiError

api_key = "sk-lhQzkeC2T6Gr6K0E6czcqg.gyPHhIECODPqvX9G9KMdSpYXRf7F2bbx".strip()



client = Sync(api_key=api_key).generations
print("Starting lip sync generation job...")

try:
    response = client.create(
        input=[Video(url=video_url), Audio(url=audio_url)],
        model="lipsync-2",
        options=GenerationOptions(sync_mode="remap"),  # best general alignment
        output_file_name="french_to_en_lipsynced"
    )
    job_id = response.id
    print("Job ID:", job_id)
except ApiError as e:
    print("[Create failed]", e.status_code, e.body)
    raise

status = None
while status not in ("COMPLETED", "FAILED"):
    time.sleep(10)
    generation = client.get(job_id)
    status = generation.status
    print("Status:", status)

print("Final status:", status)
if status == "COMPLETED":
    print(" Output:", generation.output_url)
else:
    print(" Error:", getattr(generation, "error", None))
    print("Reason:", getattr(generation, "failure_reason", None))


Starting lip sync generation job...
Job ID: b17c547b-8849-4f53-ab2c-7235c6b84110
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: PROCESSING
Status: COMPLETED
Final status: COMPLETED
 Output: https://api.sync.so/v2/generations/b17c547b-8849-4f53-ab2c-7235c6b84110/result?token=21db74f1-e22b-4d78-86f2-8bcc502f2f99
